In [ ]:
%run ../scripts/notebook_settings.py
import sgkit as sg
import xarray as xr
import glob

In [ ]:
bed_test = pd.read_csv("/home/eriks/primatediversity/data/gVCFs_recalling_10_12_2024/Papio_cynocephalus_ssp/filteredVCF/pos_bed_cov_based/Papio_cynocephalus_ssp_batch19_fploidy2_mploidy2.bed",
           sep="\t", names=["chrom", "start", "end"])
(bed_test["end"].sum()-bed_test["start"].sum())/bed_test["end"].max()

In [ ]:
# Standardized parts.
short_form = "Papio"
long_form = "Papio_papio_ssp"
size_cutoff = 1000000
window_size = 100000 # Should maybe decrease to 25 or 10kb.

metadata_path = "/home/eriks/primatediversity/data/gVCFs_recalling_10_12_2024_metadata/"
metadata_df = pd.read_csv(metadata_path+"{}_individuals.txt".format(short_form), sep="\t")
metadata_df["SEX_I"] = [0 if x == "F" else 1 for x in metadata_df.GENETIC_SEX]

In [ ]:
metadata_df

In [ ]:
sorted_f = metadata_df.loc[(metadata_df.GENETIC_SEX == "F")
                      & (metadata_df.AVG_COVERAGE_X < 40)].sort_values(by="PROP_COVERED_A", ascending=False)
sorted_f.loc[sorted_f.GVCF_ID == "SAMN03072807"] # Want to check a Ruaha baboon, which is PD_0213-217 and 503

Visualize the various PSMC curves for the test baboon cases.

In [ ]:
id_l = []
for p in glob.glob("../steps/cobraa/*b7_final_parameters.txt"):
    id_l.append(p.split("/")[-1].split("_")[0])

In [ ]:
glob.glob("../steps/cobraa/SAMEA112483032*_final_parameters.txt")

In [ ]:
mu = 0.57e-08
gen = 11
for p in glob.glob("../steps/cobraa/SAMEA112482967_*_final_parameters.txt"):
    label_tag = p.split("/")[-1].split("_")[1]
    final_params_file = p
    final_params = np.loadtxt(final_params_file)
    time_array = list(final_params[:,1])
    time_array.insert(0,0)
    time_array = np.array(time_array)
    plt.stairs(edges=(time_array/mu)*gen,values=(1/final_params[:,2])/mu,label=label_tag,linewidth=4,linestyle="solid",baseline=None)
    plt.xlim(5e+04,2e+06)
    plt.ylim(0,40e+04)
    plt.xscale('log')
    plt.ylabel('$N_A(t)$')
    plt.xlabel('Years')
    plt.tick_params(which='major',length=20)
    plt.tick_params(which='minor',length=10)
    plt.legend(ncol=2)
    plt.title('Unstructured model')
plt.show()

In [ ]:
mu = 0.57e-08
gen = 11
for p in ["../steps/cobraa/SAMEA112482967_aut_final_parameters.txt",
         "../steps/cobraa/SAMEA112482967_chrX_final_parameters.txt"]:
    label_tag = p.split("/")[-1].split("_")[1]
    final_params_file = p
    final_params = np.loadtxt(final_params_file)
    time_array = list(final_params[:,1])
    time_array.insert(0,0)
    time_array = np.array(time_array)
    plt.stairs(edges=(time_array/mu)*gen,values=(1/final_params[:,2])/mu,label=label_tag,linewidth=4,linestyle="solid",baseline=None)
    plt.xlim(5e+04,2e+06)
    plt.ylim(0,40e+04)
    plt.xscale('log')
    plt.ylabel('$N_A(t)$')
    plt.xlabel('Years')
    plt.tick_params(which='major',length=20)
    plt.tick_params(which='minor',length=10)
    plt.legend(ncol=2)
    plt.title('Unstructured model')
plt.show()

In [ ]:
mu = 0.57e-08
gen = 11
for p in glob.glob("../steps/cobraa/PD_0214_chr*_final_parameters.txt"):
    label_tag = p.split("/")[-1].split("_")[2]
    final_params_file = p
    final_params = np.loadtxt(final_params_file)
    time_array = list(final_params[:,1])
    time_array.insert(0,0)
    time_array = np.array(time_array)
    plt.stairs(edges=(time_array/mu)*gen,values=(1/final_params[:,2])/mu,label=label_tag,linewidth=4,linestyle="solid",baseline=None)
    plt.xlim(5e+04,2e+06)
    plt.ylim(0,40e+04)
    plt.xscale('log')
    plt.ylabel('$N_A(t)$')
    plt.xlabel('Years')
    plt.tick_params(which='major',length=20)
    plt.tick_params(which='minor',length=10)
    plt.legend(ncol=2)
    plt.title('Unstructured model')
plt.show()

In [ ]:
mu = 0.57e-08
gen = 11
for p in glob.glob("../steps/cobraa/SAMN02866411*_final_parameters.txt"):
    label_tag = p.split("/")[-1].split("_")[1]
    final_params_file = p
    final_params = np.loadtxt(final_params_file)
    time_array = list(final_params[:,1])
    time_array.insert(0,0)
    time_array = np.array(time_array)
    plt.stairs(edges=(time_array/mu)*gen,values=(1/final_params[:,2])/mu,label=label_tag,linewidth=4,linestyle="solid",baseline=None)
    plt.xlim(5e+04,2e+06)
    plt.ylim(0,20e+04)
    plt.xscale('log')
    plt.ylabel('$N_A(t)$')
    plt.xlabel('Years')
    plt.tick_params(which='major',length=20)
    plt.tick_params(which='minor',length=10)
    plt.legend(ncol=2)
    plt.title('Unstructured model')
plt.show()

In [ ]:
#Papio_anubis, Papio_ursinus, Papio_kindae, Papio_cynocephalus, Papio_hamadryas, Papio_papio
# SAMN02866411, SAMN02866412, SAMN03072807, SAMN02866414, SAMN24909952, SAMEA112483032
mu = 0.57e-08
gen = 11
for p in glob.glob("../steps/cobraa/SAMN03072807*_final_parameters.txt"):
    label_tag = p.split("/")[-1].split("_")[1]
    final_params_file = p
    final_params = np.loadtxt(final_params_file)
    time_array = list(final_params[:,1])
    time_array.insert(0,0)
    time_array = np.array(time_array)
    plt.stairs(edges=(time_array/mu)*gen,values=(1/final_params[:,2])/mu,label=label_tag,linewidth=4,linestyle="solid",baseline=None)
    plt.xlim(10e+04,2e+06)
    plt.ylim(0,30e+04)
    plt.xscale('log')
    plt.ylabel('$N_A(t)$')
    plt.xlabel('Years')
    plt.tick_params(which='major',length=20)
    plt.tick_params(which='minor',length=10)
    plt.legend(ncol=2)
    plt.title('Unstructured model')
plt.show()

In [ ]:
mu = 0.57e-08
gen = 11
cor_df = pd.DataFrame()
for p in glob.glob("../steps/cobraa/SAMEA112483032_*_final_parameters.txt"):
    label_tag = p.split("/")[-1].split("_")[1]
    final_params_file = p
    final_params = np.loadtxt(final_params_file)
    time_array = list(final_params[:,1])
    time_array.insert(0,0)
    time_array = np.array(time_array)
    plt.stairs(edges=(time_array/mu)*gen,values=(1/final_params[:,2])/mu,label=label_tag,linewidth=4,linestyle="solid",baseline=None)
    plt.xlim(10e+04,2e+06)
    plt.ylim(0,30e+04)
    plt.xscale('log')
    plt.ylabel('$N_A(t)$')
    plt.xlabel('Years')
    plt.tick_params(which='major',length=20)
    plt.tick_params(which='minor',length=10)
    plt.legend(ncol=2)
    plt.title('Unstructured model')
    cor_df[label_tag] = (1/final_params[:,2])/mu
plt.show()

In [ ]:
sns.heatmap(cor_df.corr())

In [ ]:
mu = 0.57e-08
gen = 11
cor_df = pd.DataFrame()
for p in glob.glob("../steps/cobraa/SAMN02866414*_final_parameters.txt"):
    label_tag = p.split("/")[-1].split("_")[1]
    final_params_file = p
    final_params = np.loadtxt(final_params_file)
    time_array = list(final_params[:,1])
    time_array.insert(0,0)
    time_array = np.array(time_array)
    plt.stairs(edges=(time_array/mu)*gen,values=(1/final_params[:,2])/mu,label=label_tag,linewidth=4,linestyle="solid",baseline=None)
    plt.xlim(10e+04,2e+06)
    plt.ylim(0,20e+04)
    plt.xscale('log')
    plt.ylabel('$N_A(t)$')
    plt.xlabel('Years')
    plt.tick_params(which='major',length=20)
    plt.tick_params(which='minor',length=10)
    plt.legend(ncol=2)
    plt.title('Unstructured model')
    cor_df[label_tag] = (1/final_params[:,2])/mu
plt.show()

In [ ]:
sns.heatmap(cor_df.corr())

In [ ]:
mu = 0.57e-08
gen = 11
cor_df = pd.DataFrame()
for p in glob.glob("../steps/cobraa/SAMN02866414_b1*_final_parameters.txt")[:]:
    label_tag = p.split("/")[-1].split("_")[1]
    final_params_file = p
    final_params = np.loadtxt(final_params_file)
    time_array = list(final_params[:,1])
    time_array.insert(0,0)
    time_array = np.array(time_array)
    plt.stairs(edges=(time_array/mu)*gen,values=(1/final_params[:,2])/mu,label=label_tag,linewidth=4,linestyle="solid",baseline=None)
    plt.xlim(10e+04,2e+06)
    plt.ylim(0,20e+04)
    plt.xscale('log')
    plt.ylabel('$N_A(t)$')
    plt.xlabel('Years')
    plt.tick_params(which='major',length=20)
    plt.tick_params(which='minor',length=10)
    plt.legend(ncol=2)
    plt.title('Unstructured model')
    cor_df[label_tag] = (1/final_params[:,2])/mu
plt.show()

In [ ]:
time_array